# PinokioCloud - The Cloud-Native Pinokio Alternative

## Cell 1: Repository Clone & Dependencies

This cell sets up the foundational requirements for the PinokioCloud system.

In [ ]:
# [Scaffold] Repository clone and dependency installation

import subprocess
import sys
import os
from pathlib import Path

# Clone repository if not already present
if not Path('PinokioCloud').exists():
    print("Cloning PinokioCloud repository...")
    subprocess.run(["git", "clone", "https://github.com/your-org/PinokioCloud.git"], check=True)

# Change to project directory
os.chdir('PinokioCloud')
sys.path.append(os.getcwd())

# Install required dependencies
dependencies = [
    'ipywidgets',
    'psutil',
    'requests',
    'pyngrok'
]

for dep in dependencies:
    subprocess.run([sys.executable, "-m", "pip", "install", dep], check=True)

print("✅ Repository and dependencies ready")

## Cell 2: Engine Initialization & Platform Detection Verification

This cell initializes core engines and verifies platform detection capabilities.

In [ ]:
# [Scaffold] Engine initialization and platform detection verification

from app.utils.P01_CloudDetector import CloudDetector
from app.utils.P01_PathMapper import PathMapper
from app.core.P02_ProcessManager import ProcessManager
from app.utils.P03_Translator import Translator
from app.core.P04_EnvironmentManager import EnvironmentManager
from app.core.P05_SearchEngine import SearchEngine
from app.utils.P05_AppAnalyzer import AppAnalyzer

# Initialize core engines
print("Initializing PinokioCloud engines...")

try:
    # Platform detection and path mapping
    cloud_detector = CloudDetector()
    platform_info = cloud_detector.detect_platform()
    path_mapper = PathMapper(platform_info)
    
    print(f"✅ Detected Platform: {platform_info.platform_name}")
    print(f"✅ Cloud Environment: {platform_info.is_cloud}")
    print(f"✅ Base Path: {path_mapper.get_base_path()}")
    
    # Initialize other engines
    process_manager = ProcessManager()
    translator = Translator()
    env_manager = EnvironmentManager(cloud_detector)
    search_engine = SearchEngine()
    app_analyzer = AppAnalyzer()
    
    print("✅ All engines initialized successfully")
    
except NotImplementedError as e:
    print(f"⚠️ Engine scaffold detected: {e}")
    print("This is expected during the scaffolding phase.")
except Exception as e:
    print(f"❌ Engine initialization error: {e}")
    import traceback
    traceback.print_exc()

## Cell 3: Foundational ipywidgets UI Shell

This cell creates the basic UI structure with tabs for all major functions.

In [ ]:
# [Scaffold] Basic ipywidgets UI with Tab widget and environment testing

import ipywidgets as widgets
from IPython.display import display
from datetime import datetime

# Create main UI structure
print("Building PinokioCloud User Interface...")

# Create output widgets for each tab
discover_output = widgets.Output()
library_output = widgets.Output()
terminal_output = widgets.Output()
tunnels_output = widgets.Output()

# Create placeholder content for each tab
with discover_output:
    print("🔍 Discover Tab - Application Search and Discovery")
    print("Features available:")
    print("- Intelligent application search with weighted relevance ranking")
    print("- Category and tag filtering with O(1) lookups")
    print("- Application details and pre-flight analysis")
    print("- Install button functionality")
    
    # Load application data
    try:
        if search_engine.load_data():
            print(f"✅ Loaded {len(search_engine.apps)} applications")
            print(f"✅ Available categories: {len(search_engine.get_categories())}")
            print(f"✅ Available tags: {len(search_engine.get_tags())}")
        else:
            print("⚠️ Failed to load application data")
    except Exception as e:
        print(f"❌ Error loading applications: {e}")
        import traceback
        traceback.print_exc()
    
    # Create search interface
    print("\n" + "="*50)
    print("🔍 APPLICATION SEARCH")
    print("="*50)
    
    # Search input
    search_input = widgets.Text(
        value='',
        placeholder='Search applications...',
        description='Search:',
        style={'description_width': 'initial'}
    )
    
    # Category dropdown
    category_dropdown = widgets.Dropdown(
        options=['All Categories'] + sorted(search_engine.get_categories()),
        value='All Categories',
        description='Category:',
        style={'description_width': 'initial'}
    )
    
    # Tag selection
    tag_selection = widgets.SelectMultiple(
        options=sorted(search_engine.get_tags()),
        value=[],
        description='Tags:',
        style={'description_width': 'initial'}
    )
    
    # GPU requirement filter
    gpu_filter = widgets.Checkbox(
        value=False,
        description='GPU Required Only',
        style={'description_width': 'initial'}
    )
    
    # Search button
    search_button = widgets.Button(
        description='Search',
        button_style='primary',
        tooltip='Search for applications'
    )
    
    # Results output area
    search_results_output = widgets.Output()
    
    def on_search_clicked(b):
        """Handle search button click"""
        with search_results_output:
            search_results_output.clear_output()
            
            # Build filters
            filters = {}
            
            if category_dropdown.value != 'All Categories':
                filters['category'] = category_dropdown.value
                
            if tag_selection.value:
                filters['tags'] = list(tag_selection.value)
                
            if gpu_filter.value:
                filters['gpu_required'] = True
            
            # Perform search
            query = search_input.value.strip()
            if query or filters:
                print(f"🔍 Searching for: '{query}'")
                if filters:
                    print(f"📋 Filters: {filters}")
                print("-" * 50)
                
                results = search_engine.search(query, filters=filters, limit=10)
                
                if results:
                    print(f"✅ Found {len(results)} results:")
                    print()
                    
                    for i, (app, score) in enumerate(results, 1):
                        print(f"{i}. {app.name} (Score: {score:.1f})")
                        print(f"   ID: {app.id}")
                        print(f"   Category: {app.category}")
                        print(f"   Tags: {', '.join(app.tags)}")
                        print(f"   GPU Required: {'Yes' if app.gpu_required else 'No'}")
                        if app.size_mb:
                            print(f"   Size: {app.size_mb} MB")
                        print(f"   Description: {app.description}")
                        print()
                        
                        # Install button for each result
                        install_btn = widgets.Button(
                            description=f'Install {app.name}',
                            button_style='success',
                            tooltip=f'Install {app.name}'
                        )
                        
                        def make_install_handler(app_id):
                            def handler(btn):
                                with search_results_output:
                                    print(f"🚀 Installing application {app_id}...")
                                    # This will be connected to the install manager in a future phase
                            return handler
                        
                        install_btn.on_click(make_install_handler(app.id))
                        display(install_btn)
                        print()
                else:
                    print("❌ No results found matching your criteria.")
            else:
                print("⚠️ Please enter a search query or select filters.")
    
    # Connect search button handler
    search_button.on_click(on_search_clicked)
    
    # Display search controls
    search_controls = widgets.VBox([
        search_input,
        widgets.HBox([category_dropdown, gpu_filter]),
        tag_selection,
        search_button,
        search_results_output
    ])
    
    display(search_controls)

with library_output:
    print("📚 My Library Tab - [Scaffold] Manage installed applications")
    print("Features to be implemented:")
    print("- List of installed applications")
    print("- Start/Stop controls")
    print("- Configuration management")
    print("- Uninstall functionality")
    print("- Certification badges")

with terminal_output:
    print("💻 Terminal Tab - Real-time process monitoring")
    print("Features available:")
    print("- Live output streaming from all processes")
    print("- Full tracebacks and error details")
    print("- Process PID tracking")
    print("- Installation and launch monitoring")
    
    # Add environment testing functionality
    print("\n" + "="*50)
    print("🧪 ENVIRONMENT TESTING TOOLS")
    print("="*50)
    
    # Environment name input
    env_name_input = widgets.Text(
        value='test_env',
        placeholder='Enter environment name',
        description='Env Name:',
        style={'description_width': 'initial'}
    )
    
    # Environment type selection
    env_type = widgets.RadioButtons(
        options=['Auto-detect', 'Force Conda', 'Force venv'],
        value='Auto-detect',
        description='Environment Type:',
        style={'description_width': 'initial'}
    )
    
    # Create environment button
    create_env_button = widgets.Button(
        description='Create Environment',
        button_style='success',
        tooltip='Create a new test environment'
    )
    
    # Test environment button
    test_env_button = widgets.Button(
        description='Test Environment',
        button_style='info',
        tooltip='Test environment detection and prefix'
    )
    
    # Clear output button
    clear_button = widgets.Button(
        description='Clear Output',
        button_style='warning',
        tooltip='Clear the terminal output'
    )
    
    # Environment output area
    env_output = widgets.Output()
    
    def stream_to_terminal(line):
        """Stream output to the terminal with timestamp"""
        timestamp = datetime.now().strftime("%H:%M:%S")
        with env_output:
            print(f"[{timestamp}] {line.strip()}")
    
    def on_create_env_clicked(b):
        """Handle create environment button click"""
        with env_output:
            env_output.clear_output()
            print(f"🚀 Creating environment: {env_name_input.value}")
            print(f"📋 Strategy: {env_type.value}")
            print("-" * 50)
        
        try:
            # Determine environment type based on selection
            if env_type.value == 'Force Conda':
                # Temporarily override platform detection
                original_detect = cloud_detector.detect_platform
                cloud_detector.detect_platform = lambda: type('PlatformInfo', (), {
                    'platform_name': 'local',
                    'is_cloud': False,
                    'cloud_provider': None
                })()
            elif env_type.value == 'Force venv':
                # Temporarily override platform detection to Lightning AI
                original_detect = cloud_detector.detect_platform
                cloud_detector.detect_platform = lambda: type('PlatformInfo', (), {
                    'platform_name': 'lightning',
                    'is_cloud': True,
                    'cloud_provider': 'lightning'
                })()
            else:
                original_detect = None
            
            # Create environment
            result = env_manager.create(env_name_input.value, callback=stream_to_terminal)
            
            # Restore original detection if modified
            if original_detect:
                cloud_detector.detect_platform = original_detect
            
            with env_output:
                print(f"✅ Environment creation result: {result}")
                
        except Exception as e:
            with env_output:
                print(f"❌ Error creating environment: {e}")
                import traceback
                traceback.print_exc()
    
    def on_test_env_clicked(b):
        """Handle test environment button click"""
        with env_output:
            env_output.clear_output()
            print(f"🔍 Testing environment: {env_name_input.value}")
            print("-" * 50)
        
        try:
            # Test environment detection
            env_type_result = env_manager._detect_environment_type(env_name_input.value)
            with env_output:
                print(f"📊 Detected environment type: {env_type_result}")
            
            # Test run prefix
            prefix = env_manager.get_run_prefix(env_name_input.value)
            with env_output:
                print(f"🔧 Run prefix: {prefix}")
                print("✅ Environment test completed successfully")
                
        except Exception as e:
            with env_output:
                print(f"❌ Error testing environment: {e}")
                import traceback
                traceback.print_exc()
    
    def on_clear_clicked(b):
        """Handle clear button click"""
        env_output.clear_output()
    
    # Connect button handlers
    create_env_button.on_click(on_create_env_clicked)
    test_env_button.on_click(on_test_env_clicked)
    clear_button.on_click(on_clear_clicked)
    
    # Display controls
    controls = widgets.VBox([
        widgets.HBox([env_name_input, env_type]),
        widgets.HBox([create_env_button, test_env_button, clear_button]),
        env_output
    ])
    
    display(controls)

with tunnels_output:
    print("🌐 Active Tunnels Tab - [Scaffold] Public URL management")
    print("Features to be implemented:")
    print("- List of active tunnel URLs")
    print("- Clickable links to application interfaces")
    print("- Tunnel status monitoring")
    print("- Manual tunnel termination")

# Create main tab widget
tab_widget = widgets.Tab()
tab_widget.children = [discover_output, library_output, terminal_output, tunnels_output]
tab_widget.titles = ['Discover', 'My Library', 'Terminal', 'Active Tunnels']

# Display the main interface
print("✅ PinokioCloud UI initialized")
display(tab_widget)